<a href="https://colab.research.google.com/github/ovuiproduction/EDI_1701/blob/main/SIH1701_Model_v_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -qU langchain-community faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 629.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00


In [3]:
!pip install transformers langchain pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 47.1 MB/s eta 0:00:00


In [4]:
from transformers import BertModel, BertTokenizer
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import pdfplumber
import numpy as np
import torch

In [5]:
# def pdf_loader(file_path):
#     text = ""
#     with pdfplumber.open(file_path) as pdf:
#         for page in pdf.pages:
#             text += page.extract_text()
#     return [Document(page_content=text)]

# # Load the PDF
# document = pdf_loader("/content/shankari_prasad.pdf")

In [6]:
# Split the text into smaller chunks
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=500,
#     chunk_overlap=0,
#     length_function=len,
# )
# docs = text_splitter.split_documents(document)

In [7]:
def pdf_loader(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

In [9]:
# Load the PDF
document = pdf_loader("/content/shankari_prasad.pdf")

In [10]:
import re
from langchain.text_splitter import TextSplitter
from langchain.docstore.document import Document

# Define the section headers to split on
SECTION_HEADERS = [
    "PETITIONER:",
    "RESPONDENT:",
    "DATE OF JUDGMENT:",
    "BENCH:",
    "CITATION:",
    "ACT:",
    "HEADNOTE:",
    "JUDGMENT:"
]

# Function to split text based on section headers
def split_text_by_sections(text):
    # Create a regular expression pattern to match section headers
    pattern = '|'.join(re.escape(header) for header in SECTION_HEADERS)
    sections = re.split(pattern, text, flags=re.IGNORECASE)
    # Return non-empty sections only
    return [section.strip() for section in sections if section.strip()]

In [11]:
sections = split_text_by_sections(document)
docs =  [Document(page_content=section) for section in sections]

In [14]:
docs[1]

Document(page_content='SRI SANKARI PRASAD SINGH DEO\nVs.')

In [19]:
docs[2]

Document(page_content='UNION OF INDIA AND STATE OF BIHAR(And Other Cases).')

In [18]:
docs[3]

Document(page_content='05/10/1951')

In [17]:
docs[4]

Document(page_content='SASTRI, M. PATANJALI')

In [16]:
docs[5]

Document(page_content='SASTRI, M. PATANJALI\nKANIA, HIRALAL J. (CJ)\nMUKHERJEA, B.K.\nDAS, SUDHI RANJAN\nAIYAR, N. CHANDRASEKHARA')

In [15]:
docs[6]

Document(page_content='1951 AIR 458 1952 SCR 89\nCITATOR INFO :\nF 1952 SC 252 (1,30)\nRF 1954 SC 257 (4)\nR 1959 SC 395 (28)\nE&D 1959 SC 512 (4)\nF 1965 SC 845 (20,21,23,24,25,27,33,35,38,39\nR 1965 SC1636 (25)\nO 1967 SC1643 (12,14,23,27,43,44,56,59,61,63\nRF 1973 SC1461 (16,20,27,30,32,38,39,44,46,88\nRF 1975 SC1193 (17)\nRF 1975 SC2299 (649)\nRF 1980 SC1789 (96)\nRF 1980 SC2056 (61)\nRF 1980 SC2097 (6)\nD 1981 SC 271 (19,33,42,43)\nRF 1986 SC1272 (78)\nRF 1986 SC1571 (34)\nRF 1987 SC1140 (3)')

In [20]:
docs[7]

Document(page_content='Constitution (First Amendment) Act, 1951, Arts. 31A,\n31B-Validity--Constitution of India, 1950, Arts. 13(2),\n368, 379, 392--Provisional Parliament--Power to amend\nConstitution- Constitution (Removal of Difficulties) Order\nNo. 2 of 1950--Validity --Amendment of Constitution--Proce-\ndure--Bill amended by Legislature--Amendment curtailing\nfundamental rights--Amendment affecting land--Validity of\nAmending Act.')

In [21]:
docs[8]

Document(page_content='The Constitution (First Amendment) Act, 1951, which has\ninserted, inter alia, Arts. 31A and 3lB in the Constitution\nof India is not ultra vires or unconstitutional.\nThe provisional Parliament is competent to exercise the\npower of amending the Constitution under Art. 368. The fact\nthat the said article refers to the two Houses of the Par-\nliament and the President separately and not to the Parlia-\nment, does not lead to the inference that the body which is\ninvested with the power to amend is not the Parliament but a\ndifferent body consisting of the two Houses.http://JUDIS.NIC.IN SUPREME COURT OF INDIA Page 2 of 14\nThe words "all the powers conferred by the provisions of\nthis Constitution on Parliament" in Art. 379 are not con-\nfined to such powers as could be exercised by the provision-\nal Parliament consisting of a single chamber, but are wide\nenough to include the power to amend the Constitution con-\nferred by Art. 368.\nThe Constitution (Removal 

In [13]:
len(docs)

10

In [ ]:
# Load LegalBERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')
model = BertModel.from_pretrained('nlpaueb/legal-bert-base-uncased')

In [ ]:
def get_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the mean of the last hidden states as embeddings
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    return embeddings

In [ ]:
# Generate embeddings for each document
texts = [doc.page_content for doc in docs]
embeddings = get_embeddings(texts)

In [ ]:
# Initialize FAISS index
embedding_dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dim)
faiss_index.add(np.array(embeddings, dtype=np.float32))

In [ ]:
# Create FAISS vector store
class FAISSVectorStore(FAISS):
    def __init__(self, index, documents):
        self.index = index
        self.docstore = {i: doc for i, doc in enumerate(documents)}
        self.index_to_docstore_id = {i: i for i in range(len(documents))}

In [ ]:
# Create the FAISS vector store
vector_store = FAISSVectorStore(faiss_index, [doc.page_content for doc in docs])

In [ ]:
from transformers import BertTokenizer, BertModel
import numpy as np
import torch
import faiss

# Function to get embeddings for a single query
def get_query_embedding(query, tokenizer, model):
    inputs = tokenizer(query, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).numpy()
    return embedding


In [ ]:
def get_ans(query) :
  # Generate the query embedding
  query_embedding = get_query_embedding(query, tokenizer, model)

  # Perform similarity search with FAISS
  k = 5  # Number of nearest neighbors to retrieve
  distances, indices = faiss_index.search(np.array(query_embedding, dtype=np.float32), k)

  # Retrieve the most similar documents
  most_similar_docs = [docs[i] for i in indices[0]]

  # Print out the most similar documents
  for i, doc in enumerate(most_similar_docs):
      print(f"Document {i}:")
      print(doc.page_content)
      print()

In [ ]:
# Define the query
query = "Extract the HEADNOTE from the following Supreme Court judgment"
get_ans(query)

In [ ]:
# Define the query
query = "Extract all ACT, laws, IPC sections , constitutional articles/acts, legal frameworks and legal statute mentioned in the text"
get_ans(query)

In [ ]:
# Define the query
query = "extract the BENCH from text"
get_ans(query)

In [ ]:
# Define the query
query = "extract Judgement from the text"
get_ans(query)

In [ ]:
def compare_pdfs(pdf_path1, pdf_path2):
    # Load and process the first PDF
    document1 = pdf_loader(pdf_path1)
    sections1 = split_text_by_sections(document1)
    docs1 =  [Document(page_content=section) for section in sections1]
    text1 = [doc.page_content for doc in docs1]
    embeddings1 = get_embeddings(text1)

    # Load and process the second PDF
    document2 = pdf_loader(pdf_path2)
    sections2 = split_text_by_sections(document2)
    docs2 =  [Document(page_content=section) for section in sections2]
    text2 = [doc.page_content for doc in docs2]
    embeddings2 = get_embeddings(text2)

    # Initialize FAISS index for both PDFs
    embedding_dim = embeddings1.shape[1]
    faiss_index2 = faiss.IndexFlatL2(embedding_dim)
    faiss_index2.add(np.array(embeddings2, dtype=np.float32))

     # # Compare the embeddings
    similarity_threshold = 0.5  # Adjust this threshold as needed
    total_comparisons = 0
    similar_comparisons = 0

    # for embedding in embeddings1:
    #     distances, indices = faiss_index2.search(np.array([embedding], dtype=np.float32), k=5) # Adjust k as needed
    #     for distance in distances[0]:
    #         total_comparisons += 1
    #         if distance < similarity_threshold:
    #             similar_comparisons += 1
    for i, embedding in enumerate(embeddings1):
        distances, _ = faiss_index2.search(np.array([embedding], dtype=np.float32), k=1)
        total_comparisons += 1
        print(distances[0][0])
        if distances[0][0] < similarity_threshold:
            similar_comparisons += 1

    # Calculate overall similarity percentage
    if total_comparisons > 0:
        similarity_percentage = (similar_comparisons / total_comparisons) * 100
    else:
        similarity_percentage = 0.0

    return similarity_percentage

In [ ]:
# Example usage
similarity_percentage = compare_pdfs("/content/sajjan_singh.pdf","/content/shankari_prasad.pdf")
print(f"Overall Similarity: {similarity_percentage:.2f}%")